# **Homework 3 - Convolutional Neural Network**

若有任何問題，歡迎來信至助教信箱 ntu-ml-2020spring-ta@googlegroups.com

In [1]:
# !gdown --id '19CzXudqN58R3D-1G8KeFWk8UDQwlb8is' --output food-11.zip # 下載資料集
# !unzip food-11.zip # 解壓縮

In [2]:
# Import需要的套件
import os
import numpy as np
import cv2
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import pandas as pd
from torch.utils.data import DataLoader, Dataset, ConcatDataset
# from apex import amp
import time
from torchsummary import summary

#Read image
利用 OpenCV (cv2) 讀入照片並存放在 numpy array 中

In [3]:
def readfile(path, label):
    # label 是一個 boolean variable，代表需不需要回傳 y 值
    image_dir = sorted(os.listdir(path))
    x = np.zeros((len(image_dir), 128, 128, 3), dtype=np.uint8)
    y = np.zeros((len(image_dir)), dtype=np.uint8)
    for i, file in enumerate(image_dir):
        img = cv2.imread(os.path.join(path, file))
        # resize to 128 x ? or ? x 128
        height = img.shape[0]
        width = img.shape[1]
        rate = 128 / max(height, width)
        height = int(height * rate)
        width = int(width * rate)
        img = cv2.resize(img, (width, height))
        # pad black
        # from https://blog.csdn.net/qq_20622615/article/details/80929746
        W, H = 128, 128
        top = (H - height) // 2
        bottom = (H - height) // 2
        if top + bottom + height < H:
            bottom += 1
        left = (W - width) // 2
        right = (W - width) // 2
        if left + right + width < W:
            right += 1
        img = cv2.copyMakeBorder(img, top, bottom, left, right, cv2.BORDER_CONSTANT, value=(0, 0, 0))
        # to np array
        x[i, :, :] = img
        if label:
          y[i] = int(file.split("_")[0])
    if label:
      return x, y
    else:
      return x

In [4]:
# 分別將 training set、validation set、testing set 用 readfile 函式讀進來
workspace_dir = './food-11'
print("Reading data")
train_x, train_y = readfile(os.path.join(workspace_dir, "training"), True)
print("Size of training data = {}".format(len(train_x)))
val_x, val_y = readfile(os.path.join(workspace_dir, "validation"), True)
print("Size of validation data = {}".format(len(val_x)))
test_x = readfile(os.path.join(workspace_dir, "testing"), False)
print("Size of Testing data = {}".format(len(test_x)))

Reading data
Size of training data = 9866
Size of validation data = 3430
Size of Testing data = 3347


# Dataset
在 PyTorch 中，我們可以利用 torch.utils.data 的 Dataset 及 DataLoader 來"包裝" data，使後續的 training 及 testing 更為方便。

Dataset 需要 overload 兩個函數：\_\_len\_\_ 及 \_\_getitem\_\_

\_\_len\_\_ 必須要回傳 dataset 的大小，而 \_\_getitem\_\_ 則定義了當程式利用 [ ] 取值時，dataset 應該要怎麼回傳資料。

實際上我們並不會直接使用到這兩個函數，但是使用 DataLoader 在 enumerate Dataset 時會使用到，沒有實做的話會在程式運行階段出現 error。


In [5]:
# training 時做 data augmentation
train_transform1 = transforms.Compose([
    transforms.ToPILImage(),
#     transforms.RandomChoice([
#         transforms.RandomVerticalFlip(),
#         transforms.RandomHorizontalFlip(),
#         transforms.RandomPerspective()
#     ]),
#     transforms.RandomChoice([
#         transforms.RandomAffine(10), # 隨機線性轉換
#         transforms.RandomRotation(40)
#     ]),
#     transforms.ColorJitter(), # 隨機色溫等
    transforms.ToTensor(), # 將圖片轉成 Tensor，並把數值 normalize 到 [0,1] (data normalization)
#     transforms.Normalize(
#         [77.89311144813877 / 255, 102.3587941606983 / 255, 126.59376063616554 / 255],
#         [72.80305392379675 / 255, 75.35438507973123 / 255, 79.31408066842762 / 255]
#     )
])
train_transform2 = transforms.Compose([
    transforms.ToPILImage(),
#     transforms.RandomOrder([
#         transforms.RandomChoice([
#             transforms.RandomHorizontalFlip(),
#             transforms.RandomPerspective()
#         ]),
#         transforms.RandomAffine(30), # 隨機線性轉換
#         transforms.RandomResizedCrop((128, 128), scale=(0.5, 1.0)), # 隨機子圖
#     ]),
#     transforms.RandomChoice([
#         transforms.ColorJitter(), # 隨機色溫等
#         transforms.RandomGrayscale(),
#     ]),
    transforms.ToTensor(), # 將圖片轉成 Tensor，並把數值 normalize 到 [0,1] (data normalization)
#     transforms.RandomErasing(0.2),
#     transforms.Normalize(
#         [77.89311144813877 / 255, 102.3587941606983 / 255, 126.59376063616554 / 255],
#         [72.80305392379675 / 255, 75.35438507973123 / 255, 79.31408066842762 / 255]
#     )
])
# testing 時不需做 data augmentation
test_transform = transforms.Compose([
    transforms.ToPILImage(),                                    
    transforms.ToTensor(),
#     transforms.Normalize(
#         [77.89311144813877 / 255, 102.3587941606983 / 255, 126.59376063616554 / 255],
#         [72.80305392379675 / 255, 75.35438507973123 / 255, 79.31408066842762 / 255]
#     )
])
class ImgDataset(Dataset):
    def __init__(self, x, y=None, transform=None):
        self.x = x
        # label is required to be a LongTensor
        self.y = y
        if y is not None:
            self.y = torch.LongTensor(y)
        self.transform = transform
    def __len__(self):
        return len(self.x)
    def __getitem__(self, index):
        X = self.x[index]
        if self.transform is not None:
            X = self.transform(X)
        if self.y is not None:
            Y = self.y[index]
            return X, Y
        else:
            return X

In [6]:
batch_size = 64
train_set = ConcatDataset([
    ImgDataset(train_x, train_y, train_transform1),
    ImgDataset(train_x, train_y, train_transform2)
])
val_set = ImgDataset(val_x, val_y, test_transform)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

# Model

In [7]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)
        # input 維度 [3, 128, 128]
        self.cnn = nn.Sequential(
#             nn.Conv2d(3, 128, 5, 1, 3),  # [3, 128, 128]
#             nn.BatchNorm2d(128),
#             nn.ReLU(),
            
            nn.Conv2d(3, 128, 3, 1, 1),  # [64, 128, 128]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [64, 64, 64]
            
            nn.Dropout2d(0.5),

            nn.Conv2d(128, 128, 3, 1, 1), # [128, 64, 64]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),      # [128, 32, 32]

            nn.Conv2d(128, 256, 3, 1, 1), # [256, 32, 32]
            nn.BatchNorm2d(256),
            nn.PReLU(1),
            nn.MaxPool2d(2, 2, 0),      # [256, 16, 16]
            
            nn.Dropout2d(0.1),

            nn.Conv2d(256, 512, 3, 1, 1), # [512, 16, 16]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 8, 8]
            
            nn.Conv2d(512, 512, 3, 1, 1), # [512, 8, 8]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),       # [512, 4, 4]
        )
        self.fc = nn.Sequential(
            nn.Linear(512*4*4, 512),
            nn.ReLU(),
            
            nn.Linear(512, 256),
            nn.ReLU(),
            
            nn.Linear(256, 128),
            nn.Dropout(0.3),
            nn.ReLU(),
            
            nn.Linear(128, 100),
            nn.ReLU(),
            
            nn.Linear(100, 30),
            nn.PReLU(1),

            nn.Linear(30, 11)
        )

    def forward(self, x):
        out = self.cnn(x)
        out = out.view(out.size()[0], -1)
        return self.fc(out)
    
summary(Classifier().cuda(), (3, 128, 128))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [-1, 128, 128, 128]           3,584
       BatchNorm2d-2        [-1, 128, 128, 128]             256
              ReLU-3        [-1, 128, 128, 128]               0
         MaxPool2d-4          [-1, 128, 64, 64]               0
         Dropout2d-5          [-1, 128, 64, 64]               0
            Conv2d-6          [-1, 128, 64, 64]         147,584
       BatchNorm2d-7          [-1, 128, 64, 64]             256
              ReLU-8          [-1, 128, 64, 64]               0
         MaxPool2d-9          [-1, 128, 32, 32]               0
           Conv2d-10          [-1, 256, 32, 32]         295,168
      BatchNorm2d-11          [-1, 256, 32, 32]             512
            PReLU-12          [-1, 256, 32, 32]               1
        MaxPool2d-13          [-1, 256, 16, 16]               0
        Dropout2d-14          [-1, 256,

# Training

使用 training set 訓練，並使用 validation set 尋找好的參數

In [8]:
model = Classifier().cuda()
# model = Classifier().cpu()
loss = nn.CrossEntropyLoss() # 因為是 classification task，所以 loss 使用 CrossEntropyLoss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # optimizer 使用 Adam
num_epoch = 300

# # use apex to optimize
# model, optimizer = amp.initialize(model, optimizer, opt_level="O2")

for epoch in range(num_epoch):
    epoch_start_time = time.time()
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    model.train() # 確保 model 是在 train model (開啟 Dropout 等...)
    for i, data in enumerate(train_loader):
        optimizer.zero_grad() # 用 optimizer 將 model 參數的 gradient 歸零
        train_pred = model(data[0].cuda()) # 利用 model 得到預測的機率分佈 這邊實際上就是去呼叫 model 的 forward 函數
        batch_loss = loss(train_pred, data[1].cuda()) # 計算 loss （注意 prediction 跟 label 必須同時在 CPU 或是 GPU 上）
#         train_pred = model(data[0].cpu())
#         batch_loss = loss(train_pred, data[1].cpu())
#         with amp.scale_loss(batch_loss, optimizer) as scaled_loss:
#             scaled_loss.backward() # 利用 back propagation 算出每個參數的 gradient
        batch_loss.backward() 
        optimizer.step() # 以 optimizer 用 gradient 更新參數值

        train_acc += np.sum(np.argmax(train_pred.cpu().data.numpy(), axis=1) == data[1].numpy())
        train_loss += batch_loss.item()
    
    model.eval()
    with torch.no_grad():
        for i, data in enumerate(val_loader):
            val_pred = model(data[0].cuda())
            batch_loss = loss(val_pred, data[1].cuda())
#             val_pred = model(data[0].cpu())
#             batch_loss = loss(val_pred, data[1].cpu())

            val_acc += np.sum(np.argmax(val_pred.cpu().data.numpy(), axis=1) == data[1].numpy())
            val_loss += batch_loss.item()

        #將結果 print 出來
        print('[%03d/%03d] %2.2f sec(s) Train Acc: %3.6f Loss: %3.6f | Val Acc: %3.6f loss: %3.6f' % \
            (epoch + 1, num_epoch, time.time()-epoch_start_time, \
             train_acc/train_set.__len__(), train_loss/train_set.__len__(), val_acc/val_set.__len__(), val_loss/val_set.__len__()))

[001/300] 77.69 sec(s) Train Acc: 0.244324 Loss: 0.033043 | Val Acc: 0.238484 loss: 0.033209
[002/300] 77.88 sec(s) Train Acc: 0.306254 Loss: 0.030685 | Val Acc: 0.342274 loss: 0.029555
[003/300] 77.94 sec(s) Train Acc: 0.355210 Loss: 0.028729 | Val Acc: 0.372012 loss: 0.027895
[004/300] 78.02 sec(s) Train Acc: 0.412579 Loss: 0.026301 | Val Acc: 0.444023 loss: 0.025678
[005/300] 77.98 sec(s) Train Acc: 0.471366 Loss: 0.023853 | Val Acc: 0.480758 loss: 0.024237
[006/300] 77.99 sec(s) Train Acc: 0.517636 Loss: 0.021865 | Val Acc: 0.488338 loss: 0.023791
[007/300] 78.07 sec(s) Train Acc: 0.565680 Loss: 0.019769 | Val Acc: 0.506706 loss: 0.023223
[008/300] 78.03 sec(s) Train Acc: 0.608301 Loss: 0.017772 | Val Acc: 0.508746 loss: 0.024137
[009/300] 78.01 sec(s) Train Acc: 0.652341 Loss: 0.015831 | Val Acc: 0.549563 loss: 0.022705
[010/300] 78.02 sec(s) Train Acc: 0.701449 Loss: 0.013590 | Val Acc: 0.536443 loss: 0.023708
[011/300] 78.00 sec(s) Train Acc: 0.749290 Loss: 0.011425 | Val Acc: 0

得到好的參數後，我們使用 training set 和 validation set 共同訓練（資料量變多，模型效果較好）

In [9]:
train_val_x = np.concatenate((train_x, val_x), axis=0)
train_val_y = np.concatenate((train_y, val_y), axis=0)
train_val_set = ConcatDataset([
    ImgDataset(train_val_x, train_val_y, train_transform1),
    ImgDataset(train_val_x, train_val_y, train_transform2),
])
train_val_loader = DataLoader(train_val_set, batch_size=batch_size, shuffle=True)

print([train_val_x[:,:,:,0].mean(), train_val_x[:,:,:,1].mean(), train_val_x[:,:,:,2].mean()])
print([train_val_x[:,:,:,0].std(), train_val_x[:,:,:,1].std(), train_val_x[:,:,:,2].std()])

[77.89311144813877, 102.3587941606983, 126.59376063616554]
[72.80305392379675, 75.35438507973123, 79.31408066842762]


In [10]:
model_best = Classifier().cuda()
# model_best = Classifier().cpu()
loss = nn.CrossEntropyLoss() # 因為是 classification task，所以 loss 使用 CrossEntropyLoss
optimizer = torch.optim.Adam(model_best.parameters(), lr=0.001) # optimizer 使用 Adam
num_epoch = 350

# use apex to optimize
# model_best, optimizer = amp.initialize(model_best, optimizer, opt_level="O3")

for epoch in range(num_epoch):
    epoch_start_time = time.time()
    train_acc = 0.0
    train_loss = 0.0

    model_best.train()
    for i, data in enumerate(train_val_loader):
        optimizer.zero_grad()
        train_pred = model_best(data[0].cuda())
        batch_loss = loss(train_pred, data[1].cuda())
#         train_pred = model_best(data[0].cpu())
#         batch_loss = loss(train_pred, data[1].cpu())
#         with amp.scale_loss(batch_loss, optimizer) as scaled_loss:
#             scaled_loss.backward()
        batch_loss.backward()
        optimizer.step()

        train_acc += np.sum(np.argmax(train_pred.cpu().data.numpy(), axis=1) == data[1].numpy())
        train_loss += batch_loss.item()

        #將結果 print 出來
    print('[%03d/%03d] %2.2f sec(s) Train Acc: %3.6f Loss: %3.6f' % \
      (epoch + 1, num_epoch, time.time()-epoch_start_time, \
      train_acc/train_val_set.__len__(), train_loss/train_val_set.__len__()))

[001/350] 98.20 sec(s) Train Acc: 0.271435 Loss: 0.032282
[002/350] 98.32 sec(s) Train Acc: 0.348187 Loss: 0.029012
[003/350] 98.35 sec(s) Train Acc: 0.419224 Loss: 0.026056
[004/350] 98.33 sec(s) Train Acc: 0.480370 Loss: 0.023496
[005/350] 98.20 sec(s) Train Acc: 0.534710 Loss: 0.021092
[006/350] 98.23 sec(s) Train Acc: 0.584349 Loss: 0.019017
[007/350] 98.21 sec(s) Train Acc: 0.626279 Loss: 0.017001
[008/350] 98.20 sec(s) Train Acc: 0.678550 Loss: 0.014817
[009/350] 98.26 sec(s) Train Acc: 0.722511 Loss: 0.012671
[010/350] 98.25 sec(s) Train Acc: 0.772375 Loss: 0.010487
[011/350] 98.24 sec(s) Train Acc: 0.813327 Loss: 0.008692
[012/350] 98.25 sec(s) Train Acc: 0.853452 Loss: 0.006827
[013/350] 98.25 sec(s) Train Acc: 0.885379 Loss: 0.005357
[014/350] 98.22 sec(s) Train Acc: 0.909823 Loss: 0.004370
[015/350] 98.23 sec(s) Train Acc: 0.924752 Loss: 0.003586
[016/350] 98.24 sec(s) Train Acc: 0.946149 Loss: 0.002636
[017/350] 98.21 sec(s) Train Acc: 0.952467 Loss: 0.002314
[018/350] 98.2

In [11]:
# save model
torch.save(model_best, 'model_no_aug.torch')

C:\Users\mingc\anaconda3\envs\ML\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Classifier. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


# Testing
利用剛剛 train 好的 model 進行 prediction

In [12]:
test_set = ImgDataset(test_x, transform=test_transform)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [13]:
model_best.eval()
prediction = []
with torch.no_grad():
    for i, data in enumerate(test_loader):
        test_pred = model_best(data.cuda())
#         test_pred = model_best(data.cpu())
        test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)
        for y in test_label:
            prediction.append(y)

In [14]:
#將結果寫入 csv 檔
with open("predict_no_aug.csv", 'w') as f:
    f.write('Id,Category\n')
    for i, y in  enumerate(prediction):
        f.write('{},{}\n'.format(i, y))

In [15]:
# 釋放記憶體
torch.cuda.empty_cache()